**Submitted By,** <br> 
Name: Anshara Chowdhury <br>
ID: 2021768 <br>
Name: Fahim Shahriar Eram <br>
ID: 2022523

In [19]:
import pandas as pd
from itertools import combinations 
import struct
from array import array
from os.path  import join
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from pandas.plotting import parallel_coordinates
import plotly.express as px
import random
import time

In [20]:
df = pd.read_csv("sample.csv", header=None)
df = df.fillna('')
df

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,,,,,,,,,,
1,30,31.0,32.0,,,,,,,,...,,,,,,,,,,
2,33,34.0,35.0,,,,,,,,...,,,,,,,,,,
3,36,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,...,,,,,,,,,,
4,38,39.0,47.0,48.0,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,38,39.0,47.0,48.0,281.0,475.0,706.0,845.0,1317.0,2859.0,...,,,,,,,,,,
996,32,39.0,225.0,3170.0,,,,,,,...,,,,,,,,,,
997,32,38.0,48.0,55.0,310.0,3171.0,,,,,...,,,,,,,,,,
998,32,36.0,38.0,39.0,41.0,48.0,89.0,110.0,681.0,940.0,...,,,,,,,,,,


In [21]:
total_transactions = len(df)
print('Total Number of Transactions:', total_transactions)

Total Number of Transactions: 1000


In [22]:
df['TransactionLength'] = df.apply(lambda row: sum(1 for value in row if value != ''), axis=1)
average_transaction_length = df['TransactionLength'].mean()
print("Average Transaction Length:", round(average_transaction_length, 2))

Average Transaction Length: 8.52


In [23]:
df = df.drop(columns=["TransactionLength"])

In [24]:
def apriori_gen(Lk, k):
    Ck = set()
    
    for itemset1 in Lk:
        for itemset2 in Lk:
            union_itemset = itemset1.union(itemset2)
            if len(union_itemset) == k + 1:
                Ck.add(union_itemset)
    
    pruned_Ck = set()
    for candidate in Ck:
        subsets = list(combinations(candidate, k))
        if all(frozenset(subset) in Lk for subset in subsets):
            pruned_Ck.add(candidate)
    
    return pruned_Ck

In [25]:
def find_frequent_itemsets(df, min_sup):
    itemset_counts = {}
    frequent_itemsets = set()
    
    for column in df.columns:
        for item in df[column].unique():
            if item != '':
                itemset = frozenset([item])
                itemset_counts[itemset] = itemset_counts.get(itemset, 0) + sum(df[column] == item)
    
    k = 1 
    frequent_itemsets_dict = {}
    
    while True:
        Lk = {itemset: count for itemset, count in itemset_counts.items() if count >= min_sup}
        
        if not Lk:
            break 
        
        frequent_itemsets.update(Lk.keys())
        Ck = apriori_gen(Lk.keys(), k)
        itemset_counts = {}
        
        for index, row in df.iterrows():
            for candidate in Ck:
                if candidate.issubset(set(row)):
                    itemset_counts[candidate] = itemset_counts.get(candidate, 0) + 1
        
        frequent_itemsets_dict[f'L-{k} itemsets'] = list(Lk)
        
        k += 1
    
    return frequent_itemsets_dict

In [26]:
def write_results_to_file(result_file_name, frequent_itemsets_dict):
    with open(result_file_name, 'w') as result_file:
        for key, value in frequent_itemsets_dict.items():
            result_file.write(f'{key}:\n')
            for itemset in value:
                result_file.write(f'{itemset}\n')
            result_file.write('\n')

In [27]:
min_sup = 3

start_time = time.time()
frequent_itemsets_dict = find_frequent_itemsets(df, min_sup)
write_results_to_file('result_retail.txt', frequent_itemsets_dict)
end_time = time.time()

elapsed_time = end_time - start_time

print(f"Total Elapsed Time (in seconds): {elapsed_time}")

Total Elapsed Time (in seconds): 706.8525793552399
